In [60]:
import pandas as pd

In [61]:
engagement_df = pd.read_csv('../prep/prep_engagements.csv')
feedback_df = pd.read_csv('../prep/prep_feedback.csv')
user_df = pd.read_csv('../prep/prep_users.csv')
course_df = pd.read_csv('../prep/prep_courses.csv')

Calculating Engagement Score using engagement and feedback dataset

In [62]:
engagement_score_df = pd.merge(engagement_df, feedback_df, on=['userId', 'courseId'])

# Normalize timeSpent and score based on max values
max_time_spent = engagement_score_df['timeSpent'].max()
max_score = engagement_score_df['score'].max()

# Convert difficulty to a numerical scale: 'Easy' = 1, 'Medium' = 2, 'Hard' = 3
difficulty_map = {'Easy': 1, 'Medium': 2, 'Hard': 3}
engagement_score_df['difficulty_num'] = engagement_score_df['difficulty'].map(difficulty_map)

# Convert 'interactive' to binary: 'Yes' = 1, 'No' = 0
engagement_score_df['interactive_num'] = engagement_score_df['interactive'].apply(lambda x: 1 if x == 'Yes' else 0)

# Calculate the engagement score using the formula
engagement_score_df['engagement_score'] = (0.4 * (engagement_score_df['timeSpent'] / max_time_spent)) + \
                         (0.3 * (engagement_score_df['score'] / max_score)) + \
                         (0.2 * (engagement_score_df['rating'] / 5)) + \
                         (0.1 * engagement_score_df['interactive_num'])
# Drop the column in place without creating a new DataFrame
engagement_score_df.drop(columns=['comments','difficulty_num','interactive_num','difficulty','interactive','rating'], inplace=True)


In [63]:
engagement_score_df.head()

,userId,courseId,timeSpent,score,discussions,engagement_score
0,128,238,69,1,11,0.315000
1,63,259,90,4,38,0.430000
2,194,5,150,2,38,0.670000
3,106,264,54,5,42,0.430000
4,33,111,223,0,2,0.551667


In [64]:
# Merge dataframes
report_df = pd.merge(engagement_df, feedback_df, on=['userId', 'courseId'], how='left')
report_df = pd.merge(report_df, user_df, on='userId', how='left')
report_df = pd.merge(report_df, course_df, on='courseId', how='left')

In [65]:
report_df.head()

,userId,courseId,timeSpent,score,discussions,rating,difficulty,comments,interactive,name_x,email,role,department,designation,name_y,est_duration,postedby,stack,prerequisites
0,21,79,185,4,12,NaN,NaN,NaN,NaN,Desiree Castro,dyerhannah@example.com,user,Devops,Intern,Science son seek resource answer.,97,1988-07-31,data engineering,HTML
1,53,232,106,5,0,NaN,NaN,NaN,NaN,Travis Smith,tiffanylee@example.com,user,Devops,Senior Software Engineer,Effort never.,140,2004-11-26,web development,CSS
2,79,51,211,5,31,NaN,NaN,NaN,NaN,Jennifer Glass,gonzalezkelly@example.net,user,Devops,Intern,Step increase.,128,1979-04-11,gen AI,CSS
3,191,227,123,1,12,NaN,NaN,NaN,NaN,Tara Taylor,robertmartin@example.org,user,Full-Stack,Principal Architect,Letter big game investment.,41,2015-12-16,data engineering,Basic SQL
4,200,144,113,5,43,NaN,NaN,NaN,NaN,Leah Townsend,turnermarie@example.net,user,Data Engineering,Solutions Enabler,Be president success field.,38,2021-11-04,data science,CSS


In [66]:
# ### Course-Level Aggregations ###
# 1. Total participants per course
participants_per_course = report_df.groupby('courseId').size().reset_index(name='total_participants')

# 2. Total time spent per course
total_time_per_course = report_df.groupby('courseId')['timeSpent'].sum().reset_index(name='total_time_spent')

# 3. Average rating per course
avg_rating_per_course = report_df.groupby('courseId')['rating'].mean().reset_index(name='avg_rating')

# 4. Average engagement score per course
avg_engagement_per_course = engagement_score_df.groupby('courseId')['engagement_score'].mean().reset_index(name='avg_engagement_score')

# 5. Total discussions count per course
discussions_per_course = report_df.groupby('courseId')['discussions'].sum().reset_index(name='total_discussions')

completion_rate = report_df.groupby('courseId')['timeSpent'].count() / report_df['courseId'].nunique()
# Merge all course-level insights
course_report = pd.merge(participants_per_course, total_time_per_course, on='courseId')
course_report = pd.merge(course_report, avg_rating_per_course, on='courseId')
course_report = pd.merge(course_report, avg_engagement_per_course, on='courseId')
course_report = pd.merge(course_report, discussions_per_course, on='courseId')
course_report = pd.merge(course_report, course_df[['courseId', 'name', 'stack']], on='courseId')


# Display the reports
print("### Course-Level Report ###")
# print(course_report)
course_report.head()


### Course-Level Report ###


,courseId,total_participants,total_time_spent,avg_rating,avg_engagement_score,total_discussions,name,stack
0,1,93,10859,1.666667,0.467222,2330,Likely sort address cost meet.,web development
1,3,93,11399,2.000000,0.499167,2475,Big interest stuff front.,data science
2,5,60,7159,3.666667,0.655556,1547,Five able.,web development
3,6,84,9554,5.000000,0.465833,1995,Today head leader.,gen AI
4,7,95,11082,2.750000,0.578750,2479,Movement poor.,gen AI


In [67]:
stack_popularity = course_report.groupby('stack')['total_participants'].sum()
stack_popularity.head()

stack
data engineering    4217
data science        3989
gen AI              5407
web development     3288
Name: total_participants, dtype: int64

In [68]:

# ### User-Level Aggregations ###
# 1. Total time spent by each user across all courses
total_time_per_user = report_df.groupby('userId')['timeSpent'].sum().reset_index(name='total_time_spent')

# 2. Average engagement score per user
avg_engagement_per_user = engagement_score_df.groupby('userId')['engagement_score'].mean().reset_index(name='avg_engagement_score')

# 3. Total discussions count by user
discussions_per_user = report_df.groupby('userId')['discussions'].sum().reset_index(name='total_discussions')

# Merge user-level insights
user_report = pd.merge(total_time_per_user, avg_engagement_per_user, on='userId')
user_report = pd.merge(user_report, discussions_per_user, on='userId')
user_report = pd.merge(user_report, user_df[['userId', 'name', 'department', 'designation']], on='userId')

print("\n### User-Level Report ###")
# print(user_report)
user_report.head()


### User-Level Report ###


,userId,total_time_spent,avg_engagement_score,total_discussions,name,department,designation
0,1,15077,0.389167,3122,Connie Cox,Consultancy,Software Engineer
1,2,14158,0.505833,2512,Debra Bishop,Data Engineering,Project Manager
2,4,15327,0.477857,3049,Monica Fuentes,Data Science,Senior Software Engineer
3,5,13294,0.547917,2920,Patricia Aguilar,Full-Stack,Solutions Enabler
4,6,15613,0.573333,3772,Misty Richardson,Consultancy,Solutions Consultant


In [69]:
#Department Level Aggregations

total_time_per_department = user_report.groupby('department')['total_time_spent'].sum().reset_index(name='total_time_spent')

totaldiscussions_per_department = user_report.groupby('department')['total_discussions'].sum().reset_index(name='total_discussions')

dept_report = pd.merge(total_time_per_department,totaldiscussions_per_department, on='department')

dept_report.head()

,department,total_time_spent,total_discussions
0,Consultancy,319819,68513
1,Data Engineering,442187,93608
2,Data Science,346134,69959
3,Devops,460720,96666
4,Full-Stack,460396,97144


In [70]:
course_report.to_csv('reporting_courses.csv', index=False)
user_report.to_csv('reporting_users.csv',index=False)
dept_report.to_csv('reporting_dept.csv',index=False)
engagement_score_df.to_csv('reporting_engagement.csv',index=False)
report_df.to_csv('reporting_fact.csv',index=False)